# DeZero with GPU

구글 콜랩을 이용하면 DeZero를 GPU에서 구동해볼 수 있습니다(구글 콜랩에서 GPU를 무료로 이용할 수 있습니다). 이제부터 DeZero에서 간단한 계산을 CPU와 GPU 양쪽에서 실행해보며, GPU가 속도를 얼마나 개선해주는지 살펴보겠습니다.

# DeZero 설치하기

먼저 DeZero를 설치합니다. DeZero는 [PyPI](https://pypi.org/project/dezero/)에 등록되어 있으므로 pip install dezero 명령으로 설치할 수 있습니다.

In [1]:
pip install dezero

설치가 완료되면 이어서 DeZero에서 GPU를 사용할 수 있는지 확인해봅니다.

In [0]:
import dezero
dezero.cuda.gpu_enable

True

True가 나오면 GPU를 사용할 수 있는 상태입니다.

False가 뜨면 구글 콜랩에서 GPU 설정을 손봐줘야 합니다. 다음 순서로 진행합니다.

* ‘런타임’ 메뉴에서 ‘런타임 유형 변경’을 선택합니다.
* ‘하드웨어 가속기’ 드롭다운 메뉴에서 ‘GPU’를 선택합니다.
* ‘저장’ 버튼을 눌러 설정을 완료합니다.

# Train MNIST with CPU
이어서 DeZero에서 MNIST를 학습해보겠습니다. 
우선은 CPU로 실행해보죠.

In [0]:
import time
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP

max_epoch = 5
batch_size = 100
cpu_times = []

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    cpu_times.append(elapsed_time)
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9140, time: 7.8949[sec]
epoch: 2, loss: 1.2791, time: 7.8918[sec]
epoch: 3, loss: 0.9211, time: 7.9565[sec]
epoch: 4, loss: 0.7381, time: 7.8198[sec]
epoch: 5, loss: 0.6339, time: 7.9302[sec]


# Train MNIST on GPU
다음은 GPU를 사용해볼 차례입니다.

In [0]:
gpu_times = []

# GPU mode
train_loader.to_gpu()
model.to_gpu()

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    gpu_times.append(elapsed_time)
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 0.5678, time: 1.5356[sec]
epoch: 2, loss: 0.5227, time: 1.5687[sec]
epoch: 3, loss: 0.4898, time: 1.5498[sec]
epoch: 4, loss: 0.4645, time: 1.5433[sec]
epoch: 5, loss: 0.4449, time: 1.5512[sec]


마지막으로 참고삼아 CPU와 GPU의 속도를 비교해봅니다.

In [0]:
cpu_avg_time = sum(cpu_times) / len(cpu_times)
gpu_avg_time = sum(gpu_times) / len(gpu_times)

print('CPU: {:.2f}[sec]'.format(cpu_avg_time))
print('GPU: {:.2f}[sec]'.format(gpu_avg_time))
print('GPU speedup over CPU: {:.1f}x'.format(cpu_avg_time/gpu_avg_time))

CPU: 7.90[sec]
GPU: 1.55[sec]
GPU speedup over CPU: 5.1x
